## Link Drive

In [72]:
from google.colab import drive
drive.mount('/content/gdrive')
#!ls "/content/gdrive/My Drive/Colab Notebooks/Capstone"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Libraries

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install xlrd

## Clean Verizon Data

Don't run it again once we have the cleaned file.

In [0]:
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/cleaned_data_updated_comcastcorp_sec_files.csv',sep='|',
                 usecols=['Unnamed: 0','form','findexdate','only_eng_words'])
df.to_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/comcast.csv',index=False)

## Clean Dictionary

Don't run it again once we have the cleaned file.

In [36]:
lm_dic = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/LoughranMcDonald_MasterDictionary_2016.csv')
lm_words = lm_dic[['Negative','Positive','Uncertainty','Litigious','Constraining','Superfluous','Interesting','Modal']].astype(bool)
lm_dic_clean = lm_dic[lm_words.Negative | lm_words.Positive | lm_words.Uncertainty | lm_words.Litigious | lm_words.Constraining | lm_words.Superfluous | lm_words.Interesting | lm_words.Modal]
lm_dic_clean = lm_dic_clean[['Word','Negative','Positive','Uncertainty','Litigious','Constraining','Superfluous','Interesting','Modal']]
lm_dic_clean = lm_dic_clean.reset_index(drop=True)
lm_1 = lm_dic_clean[['Negative','Positive','Uncertainty','Litigious','Constraining','Superfluous','Interesting']]
lm_2 = lm_dic_clean[['Word','Modal']]
lm_1[lm_1>0]=1
lm_export = pd.merge(lm_2, lm_1, left_index=True, right_index=True)
lm_export.to_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/lm_dic_clean.csv',index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)


## Load Dictionary

In [0]:
lm = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/lm_dic_clean.csv')

## Load Verizon SEC Filings and Map Sentiment

Don't run it again once we have the cleaned file.

In [0]:
# sentiment frequency
sec = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/fox.csv')
sec['only_eng_words'] = sec['only_eng_words'].str.upper()
print(sec.shape)

def map_sentiment(df):
  df_list = []
  result = []
  length = []
  for i in range(len(df)):
    print(i)
    cur_df = pd.DataFrame({'Word':df['only_eng_words'][i].split()})
    length.append(len(cur_df['Word']))
    merged_df = pd.merge(cur_df,lm,how='inner')
    df_list.append(merged_df)
    result.append(list(merged_df.apply(np.sum,axis = 0))[1:])
  result = [*zip(*result)]
  result_df = pd.merge(df,pd.DataFrame({
                                        'Sentiment':df_list,
                                        'Modal':result[0],
                                        'Negative':result[1],
                                        'Positive':result[2],
                                        'Uncertainty':result[3],
                                        'Litigious':result[4],
                                        'Constraining':result[5],
                                        'Superfluous':result[6],
                                        'Interesting':result[7],
                                        'Doc_length':length
                                       }),left_index=True,right_index=True)
  result_df['Modal_freq'] = result_df['Modal']/result_df['Doc_length']
  result_df['Negative_freq'] = result_df['Negative']/result_df['Doc_length']
  result_df['Positive_freq'] = result_df['Positive']/result_df['Doc_length']
  result_df['Uncertainty_freq'] = result_df['Uncertainty']/result_df['Doc_length']
  result_df['Litigious_freq'] = result_df['Litigious']/result_df['Doc_length']
  result_df['Constraining_freq'] = result_df['Constraining']/result_df['Doc_length']
  result_df['Superfluous_freq'] = result_df['Superfluous']/result_df['Doc_length']
  result_df['Interesting_freq'] = result_df['Interesting']/result_df['Doc_length']
  result_df['Negative_score'] = (result_df['Negative']-result_df['Positive'])/result_df['Doc_length']
  return(result_df[['Unnamed: 0','form','findexdate','Sentiment','Modal_freq','Negative_freq','Positive_freq',
                   'Uncertainty_freq','Litigious_freq','Constraining_freq','Superfluous_freq','Interesting_freq',
                   'Negative_score']])

sec_sentiment = map_sentiment(sec)
sec_sentiment.to_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/sentiment/fox_sentiment.csv',index=False)

## Load Moody's Rating and Sentiment

In [0]:
def load_data(company):
  moodys = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Capstone/Columbia DSIC 2018 - Moody\'s data.xlsx',sheet_name='Sheet1')
  if company == 'fox':
    mycompany = '21st Century Fox America'
    moodys = moodys.loc[moodys['Company']=='21st Century Fox America']
    moodys = moodys.sort_values("Effective Date")
    moodys = moodys.reset_index(drop=True)
    sentiment = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/sentiment/fox_sentiment.csv')    
  elif company == 'verizon':
    mycompany = 'Verizon Communications Inc'
    moodys = moodys.loc[moodys['Company']=='Verizon Communications Inc']
    moodys = moodys.sort_values("Effective Date")
    moodys = moodys.reset_index(drop=True)
    sentiment = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/sentiment/verizon_sentiment.csv')
  elif company == 'comcast':
    mycompany = 'Comcast Corp'
    moodys = moodys.loc[moodys['Company']=='Comcast Corp']
    moodys = moodys.sort_values("Effective Date")
    moodys = moodys.reset_index(drop=True)
    sentiment = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/sentiment/comcast_sentiment.csv')
  elif company == 'att':
    mycompany = 'AT&T Corp'
    moodys = moodys.loc[moodys['Company']=='AT&T Corp']
    moodys = moodys.sort_values("Effective Date")
    moodys = moodys.reset_index(drop=True)
    sentiment = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/sentiment/att_sentiment.csv')
  elif company == 'cbs':
    mycompany = 'CBS Corp'
    moodys = moodys.loc[moodys['Company']=='CBS Corp']
    moodys = moodys.sort_values("Effective Date")
    moodys = moodys.reset_index(drop=True)
    sentiment = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Capstone/sentiment/cbs_sentiment.csv')  
  return((moodys,sentiment,mycompany))

moodys, sentiment, company = load_data('cbs')

## EDA

In [0]:
def plot(df):
  
  date = df['findexdate']

  plt.figure(figsize=(20,8))
  plt.plot(date, df['Negative_freq'], 'b-', label='Negative Freq')
  plt.plot(date, df['Positive_freq'], 'r-', label='Positive Freq')
  plt.plot(date, df['Modal_freq'], 'y-', label='Modal Freq')
  plt.plot(date, df['Uncertainty_freq'], 'g-', label='Uncertainty Freq')
  plt.plot(date, df['Litigious_freq'], label='Litigious Freq')
  plt.plot(date, df['Constraining_freq'], label='Constraining Freq')
  plt.plot(date, df['Superfluous_freq'], label='Superfluous Freq')
  plt.plot(date, df['Interesting_freq'], label='Interesting Freq')
  plt.xticks(rotation=90)
  plt.title('Sentiment Frequency')
  plt.legend()

  plt.figure()

  plt.show()

## Plotly Frequency

In [61]:
# sentiment frequency
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

#import scipy
#from scipy import signal
#signal.savgol_filter

plotly.tools.set_credentials_file(username='yunwencai', api_key='9R0DAo2gnrurCju4VJQM')

def interactive_plot(moodys,df,company):

  # All
  negative_freq = go.Scatter(
      x = df['findexdate'],
      y = df['Negative_freq'],
      name = "Negative Frequency"
  )
  positive_freq = go.Scatter(
      x = df['findexdate'],
      y = df['Positive_freq'],
      name = "Positive Frequency"
  )
  modal_freq = go.Scatter(
      x = df['findexdate'],
      y = df['Modal_freq'],
      name = "Modal Frequency"
  )
  uncertainty_freq = go.Scatter(
      x = df['findexdate'],
      y = df['Uncertainty_freq'],
      name = "Uncertainty Frequency"
  )
  litigious_freq = go.Scatter(
      x = df['findexdate'],
      y = df['Litigious_freq'],
      name = "Litigious Frequency"
  )
  constraining_freq = go.Scatter(
      x = df['findexdate'],
      y = df['Constraining_freq'],
      name = "Constraining Frequency"
  )
  superfluous_freq = go.Scatter(
      x = df['findexdate'],
      y = df['Superfluous_freq'],
      name = "Superfluous Frequency"
  )
  interesting_freq = go.Scatter(
      x = df['findexdate'],
      y = df['Interesting_freq'],
      name = "Interesting Frequency"
  ) 
  negative_score = go.Scatter(
      x = df['findexdate'],
      y = df['Negative_score'],
      name = "Negative Score"
  ) 
  
  # 10-K
  negative_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Negative_freq'],
      name = "Negative Frequency"
  )
  positive_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Positive_freq'],
      name = "Positive Frequency"
  )
  modal_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Modal_freq'],
      name = "Modal Frequency"
  )
  uncertainty_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Uncertainty_freq'],
      name = "Uncertainty Frequency"
  )
  litigious_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Litigious_freq'],
      name = "Litigious Frequency"
  )
  constraining_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Constraining_freq'],
      name = "Constraining Frequency"
  )
  superfluous_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Superfluous_freq'],
      name = "Superfluous Frequency"
  )
  interesting_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Interesting_freq'],
      name = "Interesting Frequency"
  )
  negative_score_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Negative_score'],
      name = "Negative Score"
  )
  
  # 8-K
  negative_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = df[df['form']=='8-K']['Negative_freq'],
      name = "Negative Frequency"
  )
  positive_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = df[df['form']=='8-K']['Positive_freq'],
      name = "Positive Frequency"
  )
  modal_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = df[df['form']=='8-K']['Modal_freq'],
      name = "Modal Frequency"
  )
  uncertainty_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = df[df['form']=='8-K']['Uncertainty_freq'],
      name = "Uncertainty Frequency"
  )
  litigious_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = df[df['form']=='8-K']['Litigious_freq'],
      name = "Litigious Frequency"
  )
  constraining_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = df[df['form']=='8-K']['Constraining_freq'],
      name = "Constraining Frequency"
  )
  superfluous_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = df[df['form']=='8-K']['Superfluous_freq'],
      name = "Superfluous Frequency"
  )
  interesting_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = df[df['form']=='8-K']['Interesting_freq'],
      name = "Interesting Frequency"
  )
  negative_score_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = df[df['form']=='8-K']['Negative_score'],
      name = "Negative Score"
  )
  
  # 10-Q
  negative_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = df[df['form']=='10-Q']['Negative_freq'],
      name = "Negative Frequency"
  )
  positive_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = df[df['form']=='10-Q']['Positive_freq'],
      name = "Positive Frequency"
  )
  modal_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = df[df['form']=='10-Q']['Modal_freq'],
      name = "Modal Frequency"
  )
  uncertainty_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = df[df['form']=='10-Q']['Uncertainty_freq'],
      name = "Uncertainty Frequency"
  )
  litigious_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = df[df['form']=='10-Q']['Litigious_freq'],
      name = "Litigious Frequency"
  )
  constraining_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = df[df['form']=='10-Q']['Constraining_freq'],
      name = "Constraining Frequency"
  )
  superfluous_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = df[df['form']=='10-Q']['Superfluous_freq'],
      name = "Superfluous Frequency"
  )
  interesting_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = df[df['form']=='10-Q']['Interesting_freq'],
      name = "Interesting Frequency"
  )
  negative_score_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = df[df['form']=='10-Q']['Negative_score'],
      name = "Negative Score"
  )
  
  moodys_text = go.Scatter(
    x=moodys['Effective Date'],
    #y=[df.drop(['Unnamed: 0'],axis=1)._get_numeric_data().max(axis=0).max()]*moodys.shape[0],
    y=[0]*moodys.shape[0],
    mode='markers+text',
    name='Moody\'s Rating',
    text=moodys['Moody\'s Rating'],
    textposition='bottom center',
    textfont=dict(
        family='sans serif',
        size=12,
        color='#ff7f0e'
    )
  )
  
  layout = dict(
    title=company+' Sentiment Frequency',
    xaxis=dict(
        title='Date',
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                     label='YTD',
                     step='year',
                     stepmode='backward'),
                dict(count=5,
                     label='5y',
                     step='year',
                     stepmode='backward'),
                dict(count=10,
                     label='10y',
                     step='year',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    ),
    yaxis=dict(title='Frequency')
  )
  
  updatemenus = list([
    dict(active=-1,
         buttons=list([   
            dict(label = 'All Filings',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, True, True,
                                      False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      True]},
                         {'title': company+' All Filings'}]),
            dict(label = '10-Ks',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False,
                                      True, True, True, True, True, True, True, True, True,
                                      False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      True]},
                         {'title': company+' 10-Ks'}]),
            dict(label = '8-Ks',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      True, True, True, True, True, True, True, True, True,
                                      False, False, False, False, False, False, False, False, False,
                                      True]},
                         {'title': company+' 8-Ks'}]),
            dict(label = '10-Qs',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      True, True, True, True, True, True, True, True, True,
                                      True]},
                         {'title': company+' 10-Qs'}])
        ]),
      )
  ])
  
  layout['updatemenus'] = updatemenus

  data = [negative_freq, positive_freq, modal_freq, uncertainty_freq,
         litigious_freq, constraining_freq, superfluous_freq, interesting_freq,negative_score,
         negative_freq_10k, positive_freq_10k, modal_freq_10k, uncertainty_freq_10k,
         litigious_freq_10k, constraining_freq_10k, superfluous_freq_10k, interesting_freq_10k,negative_score_10k,
         negative_freq_8k, positive_freq_8k, modal_freq_8k, uncertainty_freq_8k,
         litigious_freq_8k, constraining_freq_8k, superfluous_freq_8k, interesting_freq_8k,negative_score_8k,
         negative_freq_10q, positive_freq_10q, modal_freq_10q, uncertainty_freq_10q,
         litigious_freq_10q, constraining_freq_10q, superfluous_freq_10q, interesting_freq_10q,negative_score_10q,
         moodys_text]  
  fig = dict(data=data, layout=layout)

  #plotly.offline.plot(fig, filename='fox_sentiment_analysis.html', auto_open=False)
  return py.iplot(fig, filename=company+' Sentiment Frequency')

interactive_plot(moodys,sentiment,company)

##Plotly Frequency with Smoothing

In [111]:
# sentiment frequency
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

import scipy
from scipy import signal
#signal.savgol_filter

plotly.tools.set_credentials_file(username='yunwencai', api_key='9R0DAo2gnrurCju4VJQM')

def interactive_plot(moodys,df,company):

  # All
  negative_freq = go.Scatter(
      x = df['findexdate'],
      y = signal.savgol_filter(df['Negative_freq'],19,3),
      name = "Negative Frequency"
  )
  positive_freq = go.Scatter(
      x = df['findexdate'],
      y = signal.savgol_filter(df['Positive_freq'],19,3),
      name = "Positive Frequency"
  )
  modal_freq = go.Scatter(
      x = df['findexdate'],
      y = signal.savgol_filter(df['Modal_freq'],19,3),
      name = "Modal Frequency"
  )
  uncertainty_freq = go.Scatter(
      x = df['findexdate'],
      y = signal.savgol_filter(df['Uncertainty_freq'],19,3),
      name = "Uncertainty Frequency"
  )
  litigious_freq = go.Scatter(
      x = df['findexdate'],
      y = signal.savgol_filter(df['Litigious_freq'],19,3),
      name = "Litigious Frequency"
  )
  constraining_freq = go.Scatter(
      x = df['findexdate'],
      y = signal.savgol_filter(df['Constraining_freq'],19,3),
      name = "Constraining Frequency"
  )
  superfluous_freq = go.Scatter(
      x = df['findexdate'],
      y = signal.savgol_filter(df['Superfluous_freq'],19,3),
      name = "Superfluous Frequency"
  )
  interesting_freq = go.Scatter(
      x = df['findexdate'],
      y = signal.savgol_filter(df['Interesting_freq'],19,3),
      name = "Interesting Frequency"
  ) 
  negative_score = go.Scatter(
      x = df['findexdate'],
      y = signal.savgol_filter(df['Negative_score'],19,3),
      name = "Negative Score"
  ) 
  
  # 10-K
  negative_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Negative_freq'],
      name = "Negative Frequency"
  )
  positive_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Positive_freq'],
      name = "Positive Frequency"
  )
  modal_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Modal_freq'],
      name = "Modal Frequency"
  )
  uncertainty_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Uncertainty_freq'],
      name = "Uncertainty Frequency"
  )
  litigious_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Litigious_freq'],
      name = "Litigious Frequency"
  )
  constraining_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Constraining_freq'],
      name = "Constraining Frequency"
  )
  superfluous_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Superfluous_freq'],
      name = "Superfluous Frequency"
  )
  interesting_freq_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Interesting_freq'],
      name = "Interesting Frequency"
  )
  negative_score_10k = go.Scatter(
      x = df[df['form']=='10-K']['findexdate'],
      y = df[df['form']=='10-K']['Negative_score'],
      name = "Negative Score"
  )
  
  # 8-K
  negative_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='8-K']['Negative_freq'],19,3),
      name = "Negative Frequency"
  )
  positive_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='8-K']['Positive_freq'],19,3),
      name = "Positive Frequency"
  )
  modal_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='8-K']['Modal_freq'],19,3),
      name = "Modal Frequency"
  )
  uncertainty_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='8-K']['Uncertainty_freq'],19,3),
      name = "Uncertainty Frequency"
  )
  litigious_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='8-K']['Litigious_freq'],19,3),
      name = "Litigious Frequency"
  )
  constraining_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='8-K']['Constraining_freq'],19,3),
      name = "Constraining Frequency"
  )
  superfluous_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='8-K']['Superfluous_freq'],19,3),
      name = "Superfluous Frequency"
  )
  interesting_freq_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='8-K']['Interesting_freq'],19,3),
      name = "Interesting Frequency"
  )
  negative_score_8k = go.Scatter(
      x = df[df['form']=='8-K']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='8-K']['Negative_score'],19,3),
      name = "Negative Score"
  )
  
  # 10-Q
  negative_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='10-Q']['Negative_freq'],19,3),
      name = "Negative Frequency"
  )
  positive_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='10-Q']['Positive_freq'],19,3),
      name = "Positive Frequency"
  )
  modal_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='10-Q']['Modal_freq'],19,3),
      name = "Modal Frequency"
  )
  uncertainty_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='10-Q']['Uncertainty_freq'],19,3),
      name = "Uncertainty Frequency"
  )
  litigious_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='10-Q']['Litigious_freq'],19,3),
      name = "Litigious Frequency"
  )
  constraining_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='10-Q']['Constraining_freq'],19,3),
      name = "Constraining Frequency"
  )
  superfluous_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='10-Q']['Superfluous_freq'],19,3),
      name = "Superfluous Frequency"
  )
  interesting_freq_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='10-Q']['Interesting_freq'],19,3),
      name = "Interesting Frequency"
  )
  negative_score_10q = go.Scatter(
      x = df[df['form']=='10-Q']['findexdate'],
      y = signal.savgol_filter(df[df['form']=='10-Q']['Negative_score'],19,3),
      name = "Negative Score"
  )
  
  moodys_text = go.Scatter(
    x=moodys['Effective Date'],
    #y=[df.drop(['Unnamed: 0'],axis=1)._get_numeric_data().max(axis=0).max()]*moodys.shape[0],
    y=[0]*moodys.shape[0],
    mode='markers+text',
    name='Moody\'s Rating',
    text=moodys['Moody\'s Rating'],
    textposition='bottom center',
    textfont=dict(
        family='sans serif',
        size=12,
        color='#ff7f0e'
    )
  )
  
  layout = dict(
    title=company+' Sentiment Frequency',
    xaxis=dict(
        title='Date',
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                     label='YTD',
                     step='year',
                     stepmode='backward'),
                dict(count=5,
                     label='5y',
                     step='year',
                     stepmode='backward'),
                dict(count=10,
                     label='10y',
                     step='year',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    ),
    yaxis=dict(title='Frequency')
  )
  
  updatemenus = list([
    dict(active=-1,
         buttons=list([   
            dict(label = 'All Filings',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, True, True,
                                      False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      True]},
                         {'title': company+' All Filings'}]),
            dict(label = '10-Ks',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False,
                                      True, True, True, True, True, True, True, True, True,
                                      False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      True]},
                         {'title': company+' 10-Ks'}]),
            dict(label = '8-Ks',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      True, True, True, True, True, True, True, True, True,
                                      False, False, False, False, False, False, False, False, False,
                                      True]},
                         {'title': company+' 8-Ks'}]),
            dict(label = '10-Qs',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      False, False, False, False, False, False, False, False, False,
                                      True, True, True, True, True, True, True, True, True,
                                      True]},
                         {'title': company+' 10-Qs'}])
        ]),
      )
  ])
  
  layout['updatemenus'] = updatemenus

  data = [negative_freq, positive_freq, modal_freq, uncertainty_freq,
         litigious_freq, constraining_freq, superfluous_freq, interesting_freq,negative_score,
         negative_freq_10k, positive_freq_10k, modal_freq_10k, uncertainty_freq_10k,
         litigious_freq_10k, constraining_freq_10k, superfluous_freq_10k, interesting_freq_10k,negative_score_10k,
         negative_freq_8k, positive_freq_8k, modal_freq_8k, uncertainty_freq_8k,
         litigious_freq_8k, constraining_freq_8k, superfluous_freq_8k, interesting_freq_8k,negative_score_8k,
         negative_freq_10q, positive_freq_10q, modal_freq_10q, uncertainty_freq_10q,
         litigious_freq_10q, constraining_freq_10q, superfluous_freq_10q, interesting_freq_10q,negative_score_10q,
         moodys_text]  
  fig = dict(data=data, layout=layout)

  #plotly.offline.plot(fig, filename='cbs_sentiment_analysis.html', auto_open=False)
  return py.iplot(fig, filename=company+' Sentiment Frequency')

interactive_plot(moodys,sentiment,company)

In [112]:
!ls
!mv /content/cbs_sentiment_analysis.html /content/gdrive/My\ Drive/Colab\ Notebooks/Capstone/sentiment_frequency_plot_w_smoothing

cbs_sentiment_analysis.html  gdrive  sample_data
